# Backpropagation

Since the output of a layer is determined by the weights between layers, the error resulting from units is scaled by the weights going forward through the network. Since we know the error at the output, we can use the weights to work backwards to hidden layers. Backpropagation is the "scaling back" of the output error to the previous layers to calculate the hidden layer error.

The error attributed to each *output unit* k is $\delta_k^o$, to find the error of each hidden layer output it is needed to be **scaled for the weights and the gradient descent* $h_j$:

(__1__):  $$h_j = f'(w_{ij} · x_i) $$

that is the definition of Gradient Descent for the function $f$ at the point $j$. So the new unit error $\delta_j^h$ for the hidden layer and the new gradient descent step $\Delta$ are the scaled values:

(__2__):  $$\delta_j^h = \sum W_{jk} \delta_k^o f'(h_j)  $$
(__3__):  $$\Delta_{ij} = \eta \delta_j^h x_i  $$ 

where $w_{ij}$ are the weights between the inputs and hidden layer and $x_i$ are input unit values. This form holds for however many layers there are. The weight steps are equal to the step size times the output error of the layer times the values of the inputs to that layer.

In [4]:
import numpy as np


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))


x = np.array([0.5, 0.1, -0.2])
target = 0.6
learnrate = 0.5

weights_input_hidden = np.array([[0.5, -0.6],
                                 [0.1, -0.2],
                                 [0.1, 0.7]])  # wi

weights_hidden_output = np.array([0.1, -0.3])   # W

## Forward pass
hidden_layer_input = np.dot(x, weights_input_hidden)  # h
hidden_layer_output = sigmoid(hidden_layer_input)     # a

output_layer_in = np.dot(hidden_layer_output, weights_hidden_output)  # W * a
output = sigmoid(output_layer_in)     # f(W * a)
print(output)

## Backwards pass
## TODO: Calculate error
# output - target
error = target - output

h = np.dot(x, weights_input_hidden)
a = sigmoid(h)
y_cap = sigmoid(np.dot(a, weights_hidden_output))
error = target - y_cap

# TODO: Calculate error gradient for output layer
# error * delta_unit * gradient_descent
del_err_output = error * (output * (1 - output))      # (yi - y) * prime_deriv( W * a)

# TODO: Calculate error gradient for hidden layer
del_err_hidden = np.dot(del_err_output, weights_hidden_output) * \
                 hidden_layer_output * (1 - hidden_layer_output)    # deltaO * W * a * (1 - a)   <<< sigmoid prime of a
print(del_err_hidden)
del_err_hidden_2 = np.dot(del_err_output, weights_hidden_output) * \
                   sigmoid_prime(hidden_layer_output)
    
assert del_err_hidden.all() == del_err_hidden_2.all()

# TODO: Calculate change in weights for hidden layer to output layer
delta_w_h_o = learnrate * del_err_output * hidden_layer_output  # eta * deltaO * a

# TODO: Calculate change in weights for input layer to hidden layer
delta_w_i_o = [learnrate * del_err_hidden * i for i in x]

print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_o)

0.48497343085
[ 0.00070802 -0.00204471]
Change in weights for hidden layer to output layer:
[ 0.00804047  0.00555918]
Change in weights for input layer to hidden layer:
[array([ 0.00017701, -0.00051118]), array([  3.54011093e-05,  -1.02235701e-04]), array([ -7.08022187e-05,   2.04471402e-04])]


```
+------+
| x[0] |
+----+-+                      HIDDEN LAYER             OUTPUT LAYER
     |
     |         w[0], w[1]
     |                      +----------------+      +-------------------+
     +--------------------->|        W       |      |                   |
                            | sigmoid(x * w) |      | sigmoid(a * W)    |
     +--------------------->|                +----->                    +------>  y
     |                      |       a        |      |         h         |
     |                      +----------------+      +-------------------+
+----+-+
| x[1] |
+------+
```

**Error gradient for output layer** `del_err_output`: 

(__4__): $$\delta^o = (1 - y) f'(W·a)$$

if target error is set to 1.

**Error gradient for hidden layer** `del_err_hidden`:

(__5__): $$\delta^h = W · \delta^o * f'(a)$$

**Change in weights for hidden layer to output layer** `delta_w_h_o`:

(__6__): $$\Delta W = \eta \delta^o * a$$

**Change in weights for input layer to hidden layer** `delta_w_i_o`:

(__7__): $$\Delta w_i = \eta \delta^h w_i $$

## Example

### Data preparation

In [2]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

## Code

In [21]:
import numpy as np

np.random.seed(42)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 3  # number of hidden units
epochs = 500
learnrate = 0.05

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden) # h = w * x
        hidden_activations = sigmoid(hidden_input)  # a = f(h)
        output = sigmoid(np.dot(weights_hidden_output, hidden_activations))

        ## Backward pass ##
        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate error gradient in output unit
        output_error = error * (weights_hidden_output * hidden_activations*(1-weights_hidden_output * hidden_activations))

        # TODO: propagate errors to hidden layer
        hidden_error = np.dot(weights_hidden_output, output_error) * (hidden_activations * (1-hidden_activations))

        # TODO: Update the change in weights
        del_w_hidden_output +=  output_error * hidden_activations
        del_w_input_hidden += hidden_error * x[:, None]   # add None values to adjust dimension

    # TODO: Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_activations = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_activations,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))


Train loss:  0.229409303659
Train loss:  0.230251430055   WARNING - Loss Increasing
Train loss:  0.231048286585   WARNING - Loss Increasing
Train loss:  0.231805931256   WARNING - Loss Increasing
Train loss:  0.232529286151   WARNING - Loss Increasing
Train loss:  0.233222274515   WARNING - Loss Increasing
Train loss:  0.233887980345   WARNING - Loss Increasing
Train loss:  0.234528802409   WARNING - Loss Increasing
Train loss:  0.235146591731   WARNING - Loss Increasing
Train loss:  0.235742769471   WARNING - Loss Increasing
Prediction accuracy: 0.725
